<a href="https://colab.research.google.com/github/gumballH2Oson/loud-mawt-works/blob/main/transfer_learning(fine_tuning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creating helper functions
Throughout your machine learning experiments, you'll likely come across snippets of code you want to use over and over again.

For example, a plotting function which plots a model's history object (see plot_loss_curves() below).

You could recreate these functions over and over again.

But as you might've guessed, rewritting the same functions becomes tedious.

One of the solutions is to store them in a helper script such as helper_functions.py. And then import the necesary functionality when you need it.

For example, you might write:

from helper_functions import plot_loss_curves

...

plot_loss_curves(history)
Let's see what this looks like.

In [1]:
# Get helper_functions.py script from course GitHub
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import helper functions we're going to use
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

--2023-10-25 03:37:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-10-25 03:37:54 (63.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



#lets get our data
We saw in the previous notebook that we could get great results with only 10% of the training data using transfer learning with TensorFlow Hub.

In this notebook, we're going to continue to work with smaller subsets of the data, except this time we'll have a look at how we can use the in-built pretrained models within the 'tf.keras.applications' module as well as how to fine-tune them to our own custom dataset.


In [2]:
# Get 10% of the data of the 10 classes
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data("10_food_classes_10_percent.zip")


--2023-10-25 03:37:58--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 74.125.20.207, 74.125.197.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   166MB/s    in 1.0s    

2023-10-25 03:37:59 (166 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:

# Walk through 10 percent data directory and list number of files
walk_through_dir("10_food_classes_10_percent")


There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/tra

In [4]:
# Create training and test directories
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"


In [6]:
import tensorflow as tf
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory= train_dir,
                                                                            image_size= IMG_SIZE,
                                                                            label_mode='categorical',
                                                                            batch_size=BATCH_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory= test_dir,
                                                                image_size= IMG_SIZE,
                                                                label_mode='categorical',
                                                                batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [7]:
#check out dataset class names

train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [8]:
# See an example batch of data
for images, labels in train_data_10_percent.take(1):
  print(images, labels)


tf.Tensor(
[[[[2.45571426e+02 2.55000000e+02 2.48142853e+02]
   [2.47234695e+02 2.55000000e+02 2.47545914e+02]
   [2.52127548e+02 2.54556122e+02 2.52566330e+02]
   ...
   [2.44841888e+02 1.95841888e+02 1.17270409e+02]
   [2.47714294e+02 1.99760223e+02 1.17760223e+02]
   [2.49387772e+02 2.02030624e+02 1.20030624e+02]]

  [[2.37260208e+02 2.54239807e+02 2.54857147e+02]
   [2.44571442e+02 2.54857147e+02 2.54724487e+02]
   [2.47459183e+02 2.54857147e+02 2.51489792e+02]
   ...
   [2.33046021e+02 1.83403229e+02 1.05046021e+02]
   [2.42867340e+02 1.97933701e+02 1.14933685e+02]
   [2.44928635e+02 2.03377686e+02 1.19117447e+02]]

  [[2.44005112e+02 2.51076538e+02 2.50362244e+02]
   [2.44698990e+02 2.52270416e+02 2.41127548e+02]
   [2.51744904e+02 2.54908157e+02 2.32392853e+02]
   ...
   [2.40693863e+02 1.93785736e+02 1.13739799e+02]
   [2.37142822e+02 1.95469376e+02 1.11841812e+02]
   [2.41076813e+02 2.03076813e+02 1.18505386e+02]]

  ...

  [[2.02658035e+02 2.16658035e+02 2.17658035e+02]
   [2

# Model 0: building a TLmodel using model using Keras

In [ ]:
from keras.src import Model
#1. creats a nbase model with tf.keras.applications

base_model = tf.keras.applications.EfficientNetB0(include_top= False)

#2. freeze the base Model
base_model.trainable= False

#3. create inputs into our models

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='input_layer')

# 4. If using ResNet50V2, add this to speed up convergence, remove for EfficientNetV2
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

#5. pass the im=nputs to the base_model

x= base_model(inputs)
# Check data shape after passing it to base_model
print(f"Shape after base_model: {x.shape}")

#6. Average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)

x=tf.keras.layers.GlobalAverage